In [1]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel

In [4]:
df = pd.read_csv("Dataframe_Wrangled_NoDuplicates.csv")
df.head()

,id,conversations
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom..."
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow..."
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u..."
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult..."


In [5]:
print(len(df))

23277


In [24]:
# parse conversations
text_list = []
for conversation in df["conversations"]:
    text = ""
    parsed_conversation = eval(conversation)
    for dialogue in parsed_conversation:
        text += dialogue["value"] + " "
    text_list.append(text)

# create column with the conversation text
df["text"] = text_list
        


In [29]:
df.head()


,id,conversations,text
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...


In [39]:
corpus = df["text"]

In [40]:
def preprocess_text(text):
    text = str(text)
    # remove angle brackets and all characters inside angle bracket
    a=re.compile("<.*?>")
    text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    # tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# preprocess all text
preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

In [48]:
df["preprocessed_text"] = preprocessed_text

In [49]:
df.head()

,id,conversations,text,preprocessed_text
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,rootopenvpnhomeopenvpn openvpninstallsh welcom...
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...",in java i want to replace string like this is ...
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,i have the following c function void addplayer...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,1 1como crear un rol para kuberntes y dar solo...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,how do i add multiple new columns in m for pow...


In [60]:
corpus = df["preprocessed_text"]

In [62]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]


KeyboardInterrupt: 

In [55]:
# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

In [56]:
# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [57]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, random_state=42)


In [58]:
topics = lda_model.print_topics()
for topic in topics:
    print(topic)

(0, '0.020*"de" + 0.014*"the" + 0.013*"la" + 0.009*"a" + 0.008*"en" + 0.007*"to" + 0.007*"const" + 0.007*"que" + 0.006*"y" + 0.006*"gt"')
(1, '0.070*"the" + 0.031*"to" + 0.029*"and" + 0.024*"a" + 0.020*"of" + 0.015*"in" + 0.013*"is" + 0.013*"for" + 0.011*"you" + 0.010*"that"')
